# Angad Kalra
# SN: 1005134999


# Primary Questions
* Q1: For all active fulltime NYT movie critics, how often do they correctly predict box office hits? 
* Q2: Does movie review sentiment align with whether it was a critic's pick or not?
* Q3: What characterisitcs are common among movies that do really well? Among movies that do poorly? 

# TO DO:
* Get all data into dataframes
* Go through each Q and complete
* Write Report

# Data Collection

In [28]:
import numpy as np
import pandas as pd
import requests, json

# Sample data collection code

# API urls & keys
nyt_url = "http://api.nytimes.com/svc/movies/v2"
nyt_apikey = "72aaefb1f009451e986a0e446468f649"
omdb_url = "http://www.omdbapi.com/?apikey=aa44212e"

# List of critics
url = nyt_url + "/critics/full-time.json"
critics_res = requests.get(url, params={"api-key": nyt_apikey})
critics_res = json.loads(critics_res.text)
critics = [x["display_name"] for x in critics_res["results"]]

# Table of movies + financial outcome
url = nyt_url + "/reviews/search.json"
movies = []
box_office_total = []
box_office_hit = []

for c in critics:
    c_res = requests.get(url, params={"api-key": nyt_apikey, "critics-pick": "Y", "reviewer": c})
    c_res = json.loads(c_res.text)
    
    results = c_res["results"]
    for m in movies:
        movies.append(m["display_title"])

        
    
df_dict = {"critic": pd.Series(critics), "movie_title": pd.Series(movies), 
                        "box_office_total": pd.Series(box_office_total), "box_office_hit": pd.Series(box_office_hit)}
box_office_movies = pd.DataFrame(df_dict)

# Table of movies + review sentiment


# Table of movies + characteristics + revenue



SyntaxError: invalid syntax (<ipython-input-28-951f259744fe>, line 20)

* I'm going to create a table of movies for each critic and also get the info on how much money the movies made in theatres. I will use the NYT and Box Office Mojo APIs for this info.
* For question #2, I'm going to scrape the NYT movie reviews from the url provided in the API response and create a dataset that has the movie info, the review, sentiment analysis, and whether it was a critic's pick. I will use NYT API for this info.
* For question #3, I'll create two datasets: one for movies that did well and one for movies that performed poorly. I will use NYT, OMDb, and Box Office Mojo APIs for this.

# EDA

* Q1: I'm going to create a list of picks for each critic and see how many of their picks performed well in box office. I will define a "box office hit" as a movie that made more than 100 million dollars domestic. 
* Q2: After scraping the reviews off the web, I will run sentiment analysis on the reviews using popular Python packages (i.e TextBlob) and classify them. I will then compare the results to whether or not the movie was a critic's pick. Plotting the data seems like a good idea right now because it will allow me to see if their is a somewhat sigmoid relationship between the two variables. My plot will be 2D, with the x-axis being the different sentiment results and the y-axis being binary (+1 for critic pick, 0 otherwise).
* Q3: For each table (movies that did well, movies that did poorly), I will perform something similar to a set intersection among their characteristics and see what they have in common. I will also remove any characteristics that appear in both tables. 

# Modelling

* Q1: Don't think I need any formal model here. Just going to look at accuracy for each critic.

* Q2: Can't think of a formal model for this. I think I will just plot sentiment vs critic pick and see whether it looks like a sigmoid function.

* Q3: I'm thinking of using PCA to find which "features" of movies are the most influential when it comes to determining revenue. 